<a href="https://colab.research.google.com/github/carvalheirafc/deep_learning_stuff/blob/master/neural_networks/le_net/le_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Section


In [22]:
from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers.convolutional import Convolution2D
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.utils import np_utils
from keras import backend
from imutils import paths
import cv2
import numpy as np
import os
import tensorflow as tf

## LeNet Class Definition


In [2]:
class LeNet:
  @staticmethod
  def build(n_channels, 
            rows, 
            cols, 
            n_classes, 
            activation='relu', 
            weights_path=None):
    model = Sequential()
    
    input_Shape = (rows, cols, n_channels)
    
   
    #Convolution and Pooling Layers
    model.add(Conv2D(20, kernel_size=5, activation=activation, input_shape=input_Shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Conv2D(50, kernel_size=5, activation=activation))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    #Flatten and Fully Conected layer
    model.add(Flatten())
    model.add(Dense(32))
    model.add(Activation(activation))
    
    # OutputLayer SOFTMAX activation
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))
    
    # if a weights path is supplied (inicating that the model was
    # pre-trained), then load the weights
    if weights_path is not None:
      model.load_weights(weights_path)
    
    return model

## Reading or Import Image Files.


In [3]:
root_path = 'Letras/'

data = []
labels = []
all_images_paths = sorted(list(paths.list_images(root_path)))
current = -1

try:
    for image_path in all_images_paths:
    
    
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = img_to_array(image)
        data.append(image)
    
        label = image_path.split(os.path.sep)[-2][7:]
        label = int(label)
        labels.append(label)
    
        if current != label:
            print('Loading Files into arrays...')
            print('Loading Class [{}]'.format(label))
        current = label

    print('All Files are Loaded Successfully...')

except IOError:
    print('Something went Wrong loading the files')    

Loading Files into arrays...
Loading Class [0]
Loading Files into arrays...
Loading Class [10]
Loading Files into arrays...
Loading Class [11]
Loading Files into arrays...
Loading Class [12]
Loading Files into arrays...
Loading Class [13]
Loading Files into arrays...
Loading Class [14]
Loading Files into arrays...
Loading Class [15]
Loading Files into arrays...
Loading Class [16]
Loading Files into arrays...
Loading Class [17]
Loading Files into arrays...
Loading Class [18]
Loading Files into arrays...
Loading Class [19]
Loading Files into arrays...
Loading Class [1]
Loading Files into arrays...
Loading Class [20]
Loading Files into arrays...
Loading Class [21]
Loading Files into arrays...
Loading Class [22]
Loading Files into arrays...
Loading Class [23]
Loading Files into arrays...
Loading Class [24]
Loading Files into arrays...
Loading Class [25]
Loading Files into arrays...
Loading Class [2]
Loading Files into arrays...
Loading Class [3]
Loading Files into arrays...
Loading Class [

In [4]:
save_files = False

data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

if save_files:
    try:
        np.save('char_input_data', data)
        np.save('labels', labels)
    except IOError:
        print('Error while saving the files')

## Model Configuration and Train Ru

- Epochs: 10

- Learning Rate: 1e-3

- Batch Size: 32

In [5]:
config = tf.ConfigProto()
sess = tf.Session(config=config) 
backend.set_session(sess)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=666)
cat_y_train = np_utils.to_categorical(y_train, 26)
cat_y_test = np_utils.to_categorical(y_test, 26)

In [7]:
n_epochs = 10
ini_learning_rate = 1e-3
batch_size = 32

In [8]:
with tf.Session() as sess:
    train_model = LeNet.build(n_channels=1, 
                              rows=28,
                              cols=28, 
                              n_classes=26, 
                              activation='relu', 
                              weights_path=None)

    opt = Adam(lr=ini_learning_rate, decay=ini_learning_rate / n_epochs)
    train_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    train_history = train_model.fit(x=x_train, 
                                    y=cat_y_train,
                                    batch_size=batch_size, 
                                    epochs=n_epochs,
                                    verbose=1)
    
    weights_file = 'weights_Letras.h5'
    try:
        train_model.save_weights(weights_file)
        print('File Writed as: {}'.format(weights_file))
    except IOError:
        print('Error while saving the Model weights')

sess.close()

Epoch 1/10
14649/14649 [==============================] - 4s 242us/step - loss: 1.1079 - acc: 0.6871
Epoch 2/10
14649/14649 [==============================] - 2s 137us/step - loss: 0.2630 - acc: 0.9244
Epoch 3/10
14649/14649 [==============================] - 2s 129us/step - loss: 0.1839 - acc: 0.9457
Epoch 4/10
14649/14649 [==============================] - 2s 128us/step - loss: 0.1516 - acc: 0.9568
Epoch 5/10
14649/14649 [==============================] - 2s 129us/step - loss: 0.1333 - acc: 0.9609
Epoch 6/10
14649/14649 [==============================] - 2s 130us/step - loss: 0.1143 - acc: 0.9668
Epoch 7/10
14649/14649 [==============================] - 2s 129us/step - loss: 0.1017 - acc: 0.9691
Epoch 8/10
14649/14649 [==============================] - 2s 128us/step - loss: 0.0944 - acc: 0.9707
Epoch 9/10
14649/14649 [==============================] - 2s 130us/step - loss: 0.0882 - acc: 0.9738
Epoch 10/10
14649/14649 [==============================] - 2s 129us/step - loss: 0.0840 - a

In [9]:
with tf.Session() as sess:
    test_model = LeNet.build(n_channels=1, 
                             rows=28,
                             cols=28, 
                             n_classes=26, 
                             activation='relu', 
                             weights_path=weights_file)

    pred = test_model.predict(x_test, verbose=1)
sess.close()

6279/6279 [==============================] - 0s 44us/step


In [23]:
precision, recall, f_betta, support = precision_recall_fscore_support(cat_y_test.argmax(1), pred.argmax(1), average='weighted')
accuracy = accuracy_score(cat_y_test.argmax(1), pred.argmax(1))
print('Accuracy: {}'.format(accuracy))
print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))
print('F Betta: {}'.format(f_betta))

Accuracy: 0.9714922758401019
Precision: 0.9729143965963092
Recall: 0.9714922758401019
F Betta: 0.9713250780303463
